In [ ]:
from synthesizer.dust.attenuation import PowerLaw
from synthesizer.dust.emission import Blackbody
from synthesizer.emission_model import EmissionModel
from synthesizer.grid import Grid
from unyt import Myr

grid_dir = "../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

# Can extract indivdual spectra
incident = EmissionModel(grid, label="incident", extract="incident")

# Or apply masks at extraction...
young_incident = EmissionModel(
    grid,
    label="young_incident",
    extract="incident",
    mask_attr="ages",
    mask_thresh=10 * Myr,
    mask_op="<",
)
old_incident = EmissionModel(
    grid,
    label="old_incident",
    extract="incident",
    mask_attr="ages",
    mask_thresh=10 * Myr,
    mask_op=">=",
)

# and then combine
incident = EmissionModel(
    grid, label="incident", combine=(young_incident, old_incident)
)

# You can pass an escape fraction
transmitted = EmissionModel(
    grid, label="transmitted", extract="transmitted", fesc=0.3
)

# For escaped the escape fraction can simply be inverted (but
# could do something more refined here)
escaped = EmissionModel(grid, label="escaped", extract="transmitted", fesc=0.7)

# More complex combinations are trivial without boilerplate in the main code
linecont = EmissionModel(
    grid,
    label="linecont",
    extract="linecont",
    mask_attr="ages",
    mask_thresh=10 * Myr,
    mask_op="<",
    fesc=0.7,
)  # where fesc is what fescLyA would have been
nebular_cont = EmissionModel(
    grid,
    label="nebular_continuum",
    extract="nebular_continuum",
    mask_attr="ages",
    mask_thresh=10 * Myr,
    mask_op="<",
)
nebular = EmissionModel(
    grid, label="nebular", combine=(linecont, nebular_cont)
)
reprocessed = EmissionModel(
    grid, label="reprocessed", combine=(nebular, transmitted)
)
intrinsic = EmissionModel(
    grid, label="intrinsic", combine=(nebular, transmitted, escaped)
)

# To apply dust we simply pass a dust curve and what it should be applied to
attenuated = EmissionModel(
    grid,
    label="attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=intrinsic,
)

# Or for something more complex like pacman we can include
# masks (or apply it to an already masked spectra if you made one)
young_attenuated = EmissionModel(
    grid,
    label="young_attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=reprocessed,
    mask_attr="ages",
    mask_thresh=10 * Myr,
    mask_op="<",
)
old_attenuated = EmissionModel(
    grid,
    label="old_attenuated",
    dust_curve=PowerLaw(slope=-0.67),
    apply_dust_to=reprocessed,
    mask_attr="ages",
    mask_thresh=10 * Myr,
    mask_op="<",
)
attenuated = EmissionModel(
    grid, label="attenuated", combine=(young_attenuated, old_attenuated)
)
emergent = EmissionModel(grid, label="emergent", combine=(attenuated, escaped))

# Finally you can apply dust emission
dust_emission = EmissionModel(
    grid, label="dust_emission", dust_emission_model=Blackbody
)

# And combine everything
total = EmissionModel(grid, label="total", combine=(emergent, dust_emission))

In [ ]:
print(total._children)
print(total._extract_keys)
print(total._dust_attenuation)
print(total._dust_emission)
print(total._combine_keys)
print(total._mask_keys)

In [ ]:
# Extraction of base spectra
transmitted = EmissionModel(
    grid, label="transmitted", extract="transmitted", fesc=0.3
)
linecont = EmissionModel(
    grid,
    label="linecont",
    extract="linecont",
    mask_attr="ages",
    mask_thresh=10 * Myr,
    mask_op="<",
    fesc=0.7,
)  # where fesc is what fescLyA would have been
nebular_cont = EmissionModel(
    grid,
    label="nebular_continuum",
    extract="nebular_continuum",
    mask_attr="ages",
    mask_thresh=10 * Myr,
    mask_op="<",
)

# Combine intermediates
nebular = EmissionModel(
    grid, label="nebular", combine=(linecont, nebular_cont)
)
reprocessed = EmissionModel(
    grid, label="reprocessed", combine=(nebular, transmitted)
)

# Escaped emission
escaped = EmissionModel(grid, label="escaped", extract="transmitted", fesc=0.7)

# Attenuate yound and old
young_attenuated = EmissionModel(
    grid,
    label="young_attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=reprocessed,
    mask_attr="ages",
    mask_thresh=10 * Myr,
    mask_op="<",
)
old_attenuated = EmissionModel(
    grid,
    label="old_attenuated",
    dust_curve=PowerLaw(slope=-0.67),
    apply_dust_to=reprocessed,
    mask_attr="ages",
    mask_thresh=10 * Myr,
    mask_op="<",
)

# Combine attenuated spectra and define emergent
attenuated = EmissionModel(
    grid, label="attenuated", combine=(young_attenuated, old_attenuated)
)
emergent = EmissionModel(grid, label="emergent", combine=(attenuated, escaped))

# Add a dust emission component
dust_emission = EmissionModel(
    grid, label="dust_emission", dust_emission_model=Blackbody
)

# And combine everything
total = EmissionModel(grid, label="total", combine=(emergent, dust_emission))

total.plot_emission_tree()